In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [96]:
def get_lf(path: str):
    return pl.scan_csv(path, separator=";")


entities = get_lf("data/data_for_spb_hakaton_entities1-Table 1.csv").filter(pl.col("entity_id").is_not_null()).select(
    "entity_id", "area", "type", "status", "priority", "ticket_number", "name", "estimation").collect()

ticket2id = dict(zip(entities["ticket_number"].to_list(), entities["entity_id"].to_list()))

entities = entities.select("entity_id", "area", "type", "status", "priority", "name", "estimation").with_columns(pl.col('status').replace_strict(
    ['Создано', 'Закрыто', 'Выполнено', 'Отклонен исполнителем'],
    [0, 2, 2, 2], default=1, return_dtype=pl.Int8)).with_columns(
    pl.col('type').replace_strict(
        ['Дефект'],
        [1], default=0, return_dtype=pl.Int8)
).with_columns(pl.col("estimation").fill_null(0))
entities


entity_id,area,type,status,priority,name,estimation
i64,str,i8,i8,str,str,i64
94297,"""Система.Таск-трекер""",1,2,"""Средний""","""[FE] Бэклог. Кастомизация коло…",60
102481,"""Система.Ошибки""",0,2,"""Критический""","""[ГенераторДокументов] Интеграц…",432000
1805925,"""Система.Таск-трекер""",1,1,"""Высокий""","""[FE] История изменений. Пустые…",60
1934905,"""Система.Таск-трекер""",1,2,"""Средний""","""[FE] Зависимые поля. Тип реакц…",0
1943849,"""Система.Ошибки""",1,2,"""Низкий""","""[BE] При сортировке по Теме ес…",0
…,…,…,…,…,…,…
5179477,"""Управление релизами изменениям…",0,2,"""Средний""","""[PPPL] [FE] - Исследовать ui-k…",57600
5179714,"""Управление релизами изменениям…",0,2,"""Средний""","""[PPPL] [FE] - Написать базовую…",86400
5179793,"""Управление релизами изменениям…",0,2,"""Средний""","""[PPPL] [FE] - Сделать роутинг…",28800


In [ ]:
sprints = get_lf("data/sprints-Table 1.csv").filter(pl.col("sprint_name").is_not_null()).select(
    "sprint_name","sprint_start_date", "entity_ids").with_columns(pl.col("sprint_start_date").str.to_datetime().dt.timestamp("ms")//86400000).collect()

tasks = sprints.with_columns(pl.col("entity_ids").str.strip_chars(r'{}').str.split(",").alias("entity_id")).explode("entity_id")
tasks = tasks.with_columns(pl.col("entity_id").cast(pl.Int64, strict=False))
all_tasks_estimations = entities.select("entity_id", "estimation").join(tasks.select("entity_id", "sprint_name"), on="entity_id").group_by("sprint_name").agg(pl.col("estimation").sum())


# for i in sprints.rows_by_key("sprint_name").items():
#     sprints = 
sprints = {k:(i,v) for i,(k,v) in enumerate(zip(sprints["sprint_name"].to_list(), sprints["sprint_start_date"].to_list()))}
from enum import IntEnum

class Feature(IntEnum):
    to_do_estimation=0
    processed_estimation=1
    done_estimation=2
    removed_estimation=3
    blocked_tasks_estimation=4
    created_tasks_estimation=5
    excluded_tasks_estimation=6
    created_tasks_amount=7
    excluded_tasks_amount=8

day_features = np.zeros((len(sprints), 14, len(Feature._member_names_)))

In [102]:
all_tasks_estimations

sprint_name,estimation
str,i64
"""Спринт 2024.3.4.NPP Shared Spr…",15811200
"""Спринт 2024.3.2.NPP Shared Spr…",17067720
"""Спринт 2024.3.3.NPP Shared Spr…",17776920
"""Спринт 2024.3.1.NPP Shared Spr…",18338580
"""Спринт 2024.3.5.NPP Shared Spr…",17784060
"""Спринт 2024.3.6.NPP Shared Spr…",20214060


In [3]:
# entities
state2int = {"created":0, "closed":2, "done":2, "rejectedByThePerformer":2}
entities = {k:list(v[0])+["", ""] for k,v in entities.rows_by_key("entity_id").items()}


In [4]:
import numpy as np

In [ ]:
q = '''
SELECT COUNT
'''

In [ ]:
sprints["Спринт 2024.3.1.NPP Shared Sprint"]

In [73]:
sprints

sprint_name,sprint_start_date,entity_ids,entity_id
str,i64,str,u32
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",4449728
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",4450628
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",4451563
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",4451929
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",4452033
…,…,…,…
"""Спринт 2024.3.6.NPP Shared Spr…",19977,"""{5034080,5052223,4620977,51371…",5068008
"""Спринт 2024.3.6.NPP Shared Spr…",19977,"""{5034080,5052223,4620977,51371…",5030342
"""Спринт 2024.3.6.NPP Shared Spr…",19977,"""{5034080,5052223,4620977,51371…",5070230


In [71]:
entities[94297]

['Система.Таск-трекер',
 1,
 2,
 'Средний',
 '[FE] Бэклог. Кастомизация колонок. Кастомизация для панелей "спринты" и  "бэклоги" зависима друг от друга',
 60,
 '',
 '']

In [8]:
entities

{94297: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Бэклог. Кастомизация колонок. Кастомизация для панелей "спринты" и  "бэклоги" зависима друг от друга',
  60,
  '',
  ''],
 102481: ['Система.Ошибки',
  0,
  2,
  'Критический',
  '[ГенераторДокументов] Интеграция со Система.ГенераторДокументов в части синхронизации Стандартных решений',
  432000,
  '',
  ''],
 1805925: ['Система.Таск-трекер',
  1,
  1,
  'Высокий',
  '[FE] История изменений. Пустые строки в истории изменений, если выбрать значение, которое уже было выбрано до редактирования поля',
  60,
  '',
  ''],
 1934905: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Зависимые поля. Тип реакции disable НЕ работает для некоторых типов полей',
  0,
  '',
  ''],
 1943849: ['Система.Ошибки',
  1,
  2,
  'Низкий',
  '[BE] При сортировке по Теме если знаки препинания стоят перед буквенными значениями сортировка отрабатывает неправильно',
  0,
  '',
  ''],
 1966759: ['Управление релизами изменениями',
  1,
  1,
  'Низ

In [9]:
history = get_lf("data/history-Table 1.csv").filter(
    pl.col("entity_id").is_not_null()
    & pl.col("history_property_name").is_not_null()
    & pl.col("history_property_name").is_in(["Статус", "Резолюция", "Спринт", "Связанные Задачи"])
).select(
    "entity_id", "history_property_name", "history_date", "history_change").collect()
history = history.with_columns(
    pl.col("history_date").str.to_datetime('%m/%d/%y %H:%M', time_unit="ms", strict=False)).with_columns(
    pl.col("history_change").str.split(" -> ").list.last()).sort(
    'history_date', maintain_order=True).with_columns(
    pl.col('history_change').shift(1).alias('history_change_prev')).with_columns(
    pl.col('entity_id').shift(1).alias('entity_id_prev')).with_columns(
    (pl.col("history_date").dt.timestamp("ms") // 86400000).alias("day"))

In [10]:
history.filter(pl.col("history_property_name") == "Статус")["history_change"].unique().to_list()

['closed',
 'review',
 'created',
 'localization',
 'inProgress',
 'development',
 'at',
 'design',
 'waiting',
 'done',
 'stCompleted',
 'analysis',
 'introduction',
 'testing',
 'rejectedByThePerformer',
 'readyForDevelopment',
 'hold',
 'fixing',
 'st',
 'ift',
 'verification']

In [11]:
i = 0
h = history.filter(pl.col("entity_id")==3864586)
result = []
for day, df in h.group_by("day", maintain_order=True):
    print(df)
    df = df.group_by("history_property_name", maintain_order=True).last()
    for row in df.iter_rows():
        ent_id = row[1]
        if ent_id not in entities:
            continue
        match row[0]:
            case "Резолюция":
                entities[ent_id][6] = row[3]
                break
            case "Статус":
                new_state = state2int.get(row[3],2)
                ent = entities[ent_id]
                ent[3] = new_state
                sprint_name = ent[7]
                sprint = sprints.get(sprint_name, None)
                if sprint is not None:
                    sprint_id = sprint[0]
                    day_offset = sprint[1]-day
                    if day_offset<0:
                        day_offset=0
                    elif day_offset>13:
                        continue
                    if new_state==0:
                        day_features[sprint_id,day_offset, Feature.to_do_estimation]+=ent[5]
                break
            case "Спринт":
                break
        if row[0]=="Статус":
            pass
        print(day, row)
    # if df.shape[0]>1:
    #     i += 1
        # print(df)
    # if i == 10:
    #     break

shape: (14, 7)
┌───────────┬───────────────┬───────────────┬───────────────┬───────────────┬──────────────┬───────┐
│ entity_id ┆ history_prope ┆ history_date  ┆ history_chang ┆ history_chang ┆ entity_id_pr ┆ day   │
│ ---       ┆ rty_name      ┆ ---           ┆ e             ┆ e_prev        ┆ ev           ┆ ---   │
│ i64       ┆ ---           ┆ datetime[ms]  ┆ ---           ┆ ---           ┆ ---          ┆ i64   │
│           ┆ str           ┆               ┆ str           ┆ str           ┆ i64          ┆       │
╞═══════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪══════════════╪═══════╡
│ 3864586   ┆ Связанные     ┆ 2024-04-24    ┆ isClonedBy    ┆ PPIN-3139     ┆ 2824925      ┆ 19837 │
│           ┆ Задачи        ┆ 12:30:00      ┆               ┆               ┆              ┆       │
│ 3864586   ┆ Связанные     ┆ 2024-04-24    ┆ PPIN-3080     ┆ isClonedBy    ┆ 3864586      ┆ 19837 │
│           ┆ Задачи        ┆ 12:30:00      ┆               ┆               

In [12]:
h

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
3864586,"""Связанные Задачи""",2024-04-24 12:30:00,"""isClonedBy""","""PPIN-3139""",2824925,19837
3864586,"""Связанные Задачи""",2024-04-24 12:30:00,"""PPIN-3080""","""isClonedBy""",3864586,19837
3864586,"""Связанные Задачи""",2024-04-24 12:32:00,null,"""PPIN-3080""",3864586,19837
3864586,"""Связанные Задачи""",2024-04-24 12:33:00,"""clones""",null,2824925,19837
3864586,"""Связанные Задачи""",2024-04-24 12:33:00,"""PPIN-3143""","""clones""",3864586,19837
…,…,…,…,…,…,…
3864586,"""Связанные Задачи""",2024-08-05 08:46:00,"""clones""","""inProgress""",4605512,19940
3864586,"""Связанные Задачи""",2024-08-05 08:46:00,"""PPIN-3471""","""clones""",3864586,19940
3864586,"""Резолюция""",2024-08-21 11:39:00,"""Готово""","""inProgress""",4755587,19956


In [13]:
history = history.with_columns(pl.col("history_change").str.split(" -> ").list.last())
# history
history

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
1805925,"""Спринт""",2023-07-12 13:29:00,"""Спринт 2023.3.2.NPP Shared Spr…",null,null,19550
1805925,"""Спринт""",2023-07-31 17:43:00,null,"""Спринт 2023.3.2.NPP Shared Spr…",1805925,19569
1805925,"""Спринт""",2023-07-31 17:43:00,"""Спринт 2023.3.3.NPP Shared Spr…",null,1805925,19569
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""8""","""Спринт 2023.3.3.NPP Shared Spr…",1805925,19573
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""PPTS-3254""","""8""",1934905,19573
…,…,…,…,…,…,…
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.4.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.2.NPP Shared Spr…","""Спринт 2024.4.1.NPP Shared Spr…",4707481,20033


In [14]:
history.filter(~pl.col("history_change").str.contains("->"))

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
1805925,"""Спринт""",2023-07-12 13:29:00,"""Спринт 2023.3.2.NPP Shared Spr…",null,null,19550
1805925,"""Спринт""",2023-07-31 17:43:00,"""Спринт 2023.3.3.NPP Shared Spr…",null,1805925,19569
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""8""","""Спринт 2023.3.3.NPP Shared Spr…",1805925,19573
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""PPTS-3254""","""8""",1934905,19573
102481,"""Связанные Задачи""",2023-08-17 14:11:00,"""29""","""PPTS-3254""",1934905,19586
…,…,…,…,…,…,…
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.4.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.2.NPP Shared Spr…","""Спринт 2024.4.1.NPP Shared Spr…",4707481,20033


In [15]:
history.with_columns(
    pl.col("history_change").arr.get(0)
)


SchemaError: invalid series dtype: expected `FixedSizeList`, got `str`

In [ ]:
history.filter(history["history_date"].is_null())

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64


In [ ]:
df = pd.read_csv("data/history-Table 1.csv", sep=';')
df = df[df['entity_id'].notna()]
#df = df[df['history_change_type'].notna()]
df['entity_id'] = df['entity_id'].astype(np.int32)
df['history_version'] = df["history_version"].astype(np.int32)
df = df.drop(["Столбец1", "Unnamed: 7"], axis=1)

In [ ]:
df['history_property_name'].unique()

array(['Время решения 3ЛП ФАКТ', 'Время решения (ФАКТ)', 'Исполнитель',
       'Срок исполнения', 'Спринт', 'Статус', 'Резолюция', 'Метки',
       'Дата решения', 'Задача', nan, 'Название', 'Приоритет',
       'Стрим-владелец', 'Стрим-заказчик', 'Проект-заказчик',
       'Родительская задача', 'Связанные Задачи', 'Дочерние Задачи',
       'Владелец', 'Оценка', 'Релиз', 'Копирование', 'Компоненты',
       'Story Points', 'Учет рабочего времени', 'Код системы', 'Системы',
       'Причины Арх. Таск-трекер', 'Тип работ', 'Системы обнаружения',
       'Обнаружено автотестом', 'Время решения 4ЛП ПЛАН',
       'Время решения 4ЛП ФАКТ', 'Время Исправления (ПЛАН)',
       'Время Исправления (ФАКТ)', 'Причина отсутствия НТ', 'Вложения',
       'Среда обнаружения', 'Согласие на передачу прав',
       'Затронуты релизы', 'Время решения (ПЛАН)', 'Подстатус',
       'Код системы обнаружения', 'Осталось', 'Комментарий при решении',
       'Причины дефектов ПРОМ', 'Время восстановления сервиса ПЛАН',


In [ ]:
df[df['history_property_name'] == "Спринт"]

,entity_id,history_property_name,history_date,history_version,history_change_type,history_change
7,94297,Спринт,9/10/24 11:16,6,FIELD_CHANGED,<empty> -> Спринт 2024.3.5.NPP Shared Sprint
115,102481,Спринт,5/16/24 7:59,81,FIELD_CHANGED,<empty> -> Суперспринт 2024.2.NPP Shared Sprint
124,102481,Спринт,7/15/24 8:30,88,FIELD_CHANGED,Суперспринт 2024.2.NPP Shared Sprint -> Спринт...
125,102481,Спринт,7/15/24 8:30,88,FIELD_CHANGED,Суперспринт 2024.2.NPP Shared Sprint -> Суперс...
163,1805925,Спринт,7/12/23 13:29,23,FIELD_CHANGED,<empty> -> Спринт 2023.3.2.NPP Shared Sprint
...,...,...,...,...,...,...
64172,5179881,Спринт,10/23/24 7:00,11,FIELD_CHANGED,Спринт 2024.3.6.NPP Shared Sprint -> Спринт 20...
64173,5179881,Спринт,10/23/24 7:00,11,FIELD_CHANGED,Спринт 2024.3.6.NPP Shared Sprint -> Спринт 20...
64174,5179881,Спринт,10/23/24 7:00,11,FIELD_CHANGED,Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...
64175,5179881,Спринт,10/23/24 7:00,11,FIELD_CHANGED,Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...


In [ ]:
df[df['history_change_type'] == "CREATED"]['history_property_name'].unique()

array(['Задача'], dtype=object)

In [ ]:
def custom_fn(x):
    return x


df.groupby(['entity_id', 'history_version']).apply(custom_fn)

entity_id   history_property_name  \
entity_id history_version                                            
94297     1               0          94297  Время решения 3ЛП ФАКТ   
                          1          94297    Время решения (ФАКТ)   
                          2          94297             Исполнитель   
          3               4          94297             Исполнитель   
          4               5          94297         Срок исполнения   
...                                    ...                     ...   
5179881   11              64174    5179881                  Спринт   
                          64175    5179881                  Спринт   
                          64176    5179881                  Спринт   
          12              64177    5179881               Резолюция   
                          64178    5179881                  Статус   

                                  history_date  history_version  \
entity_id history_version                                         
94297     1               0      9/10/24 11:17                1   
                          1      9/10/24 11:17                1   
                          2      7/13/23 11:07                1   
          3               4      7/21/23 11:06                3   
          4               5      7/31/23 15:05                4   
...                                        ...              ...   
5179881   11              64174  10/23/24 7:00               11   
                          64175  10/23/24 7:00               11   
                          64176  10/23/24 7:00               11   
          12              64177  10/23/24 7:00               12   
                          64178  10/23/24 7:00               12   

                                history_change_type  \
entity_id history_version                             
94297     1               0           FIELD_CHANGED   
                          1           FIELD_CHANGED   
                          2           FIELD_CHANGED   
          3               4           FIELD_CHANGED   
          4               5           FIELD_CHANGED   
...                                             ...   
5179881   11              64174       FIELD_CHANGED   
                          64175       FIELD_CHANGED   
                          64176       FIELD_CHANGED   
          12              64177       FIELD_CHANGED   
                          64178       FIELD_CHANGED   

                                                                    history_change  
entity_id history_version                                                           
94297     1               0                  <empty> -> 2024-09-10 11:17:06.680223  
                          1                  <empty> -> 2024-09-10 11:17:06.680223  
                          2      user409017mail@mail.com -> user408045mail@mail...  
          3               4           user4080458@mail.com -> user4091341@mail.com  
          4               5                                                    NaN  
...                                                                            ...  
5179881   11              64174  Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...  
                          64175  Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...  
                          64176  Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...  
          12              64177                                  <empty> -> Готово  
                          64178                               inProgress -> closed  

[61120 rows x 6 columns]

In [ ]:
df = pd.read_csv("data/data_for_spb_hakaton_entities1-Table 1.csv", sep=';')
df

,entity_id,area,type,status,state,priority,ticket_number,name,create_date,created_by,...,updated_by,parent_ticket_id,assignee,owner,due_date,rank,estimation,spent,workgroup,resolution
0,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,А. К.,72779.0,А. К.,А. К.,NaN,0|qzzywk:,60.0,NaN,NaN,Готово
1,102481,Система.Ошибки,История,Закрыто,Normal,Критический,PPIN-1175,[ГенераторДокументов] Интеграция со Система.Ге...,2023-05-12 13:33:55.918127,А. З.,...,NaN,3488105.0,А. Е.,А. Е.,NaN,0|qv7n1c:y,432000.0,NaN,Новая функциональность,Готово
2,1805925,Система.Таск-трекер,Дефект,Тестирование,Normal,Высокий,PPTS-3189,[FE] История изменений. Пустые строки в истори...,2023-07-12 09:36:04.479760,А. К.,...,А. К.,NaN,А. К.,А. К.,NaN,0|qzsklw:,60.0,NaN,NaN,NaN
3,1934905,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-3383,[FE] Зависимые поля. Тип реакции disable НЕ ра...,2023-08-04 11:32:25.829919,А. Д.,...,А. К.,NaN,А. К.,Я. П.,NaN,0|qzh3e8:,NaN,NaN,NaN,Готово
4,1943849,Система.Ошибки,Дефект,Отклонен исполнителем,Normal,Низкий,PPIN-1609,[BE] При сортировке по Теме если знаки препина...,2023-08-09 06:20:44.391950,А. С.,...,А. Е.,2083371.0,А. М.,А. Ж.,NaN,0|qzzywj:zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz...,NaN,NaN,NaN,Отклонено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,5179477,Управление релизами изменениями,Задача,Закрыто,Normal,Средний,PPRC-2511,[PPPL] [FE] - Исследовать ui-kit andt на приме...,2024-09-24 12:12:21.879096,Е. Б.,...,В. М.,4933112.0,В. М.,Е. Б.,NaN,0|qm0tow:,57600.0,NaN,Новая функциональность,NaN
2481,5179714,Управление релизами изменениями,Задача,Закрыто,Normal,Средний,PPRC-2512,[PPPL] [FE] - Написать базовую структуру прило...,2024-09-24 12:18:13.307422,Е. Б.,...,В. М.,4933112.0,В. М.,Е. Б.,NaN,0|qm0sbk:,86400.0,NaN,Новая функциональность,Готово
2482,5179793,Управление релизами изменениями,Задача,Закрыто,Normal,Средний,PPRC-2513,[PPPL] [FE] - Сделать роутинги в админке плаг...,2024-09-24 12:19:51.036183,Е. Б.,...,В. М.,4933112.0,В. М.,Е. Б.,NaN,0|qm0rw0:,28800.0,NaN,Новая функциональность,Готово
2483,5179847,Управление релизами изменениями,Задача,Закрыто,Normal,Средний,PPRC-2515,[PPPL] [FE] - Написать парсинга json файла,2024-09-24 12:21:27.373853,Е. Б.,...,И. К.,4933112.0,В. М.,Е. Б.,NaN,0|qm0rko:,28800.0,NaN,Новая функциональность,Готово


In [ ]:
for col in ["type", "status", "state", "priority", "resolution"]:
    a = map(lambda x: f"'{x}'", df[col].unique())
    print(f"{col} Enum8({','.join(a)})")

type Enum8('Дефект','История','Задача','Подзадача')
status Enum8('Закрыто','Тестирование','Отклонен исполнителем','Исправление','Анализ','Создано','Выполнено','Отложен','В работе','Готово к разработке','СТ','Подтверждение исправления','Разработка','Подтверждение','СТ Завершено','Локализация','В ожидании')
state Enum8('Normal')
priority Enum8('Средний','Критический','Высокий','Низкий')
resolution Enum8('Готово','nan','Отклонено','Отменен инициатором','Дубликат')
